In [ ]:
import os

In [ ]:
os.getcwd()

In [ ]:
os.chdir("..")

In [ ]:
os.getcwd()

# config

##### raw/config.yaml

# constants

##### schema

In [ ]:
# update __all__ 

from pydantic import BaseModel, Field 
from datetime import datetime


class Constants:
    ...

class DataIngestionConstants(BaseModel):
    TIME_STAMP: datetime
    ROOT_DIR_NAME: str = Field(frozen=True) 
    DATA_ROOT_DIR_NAME: str = Field(frozen=True) 
    INGESTION_ROOT_DIR_NAME: str = Field(frozen=True) 
    RAW_DATA_DIR_NAME: str = Field(frozen=True)
    OUTPUT_DIR_NAME: str = Field(frozen=True)

class DataTransformationConstants(BaseModel):
    PROMPT: str
    TIME_STAMP: datetime
    ROOT_DIR_NAME: str = Field(frozen=True) 
    DATA_ROOT_DIR_NAME: str = Field(frozen=True) 
    TRANSFORMATION_ROOT_DIR_NAME: str = Field(frozen=True) 
    PARSED_DATA_DIR_NAME: str = Field(frozen=True)
    STRUCTURED_DATA_DIR_NAME: str = Field(frozen=True)
    TRAIN_DATA_DIR_NAME: str = Field(frozen=True)
    OUTPUT_DIR_NAME: str = Field(frozen=True)

class JobDescriptionConstants(BaseModel):
    TIME_STAMP: datetime
    ROOT_DIR_NAME: str = Field(frozen=True) 
    JD_ROOT_DIR_NAME: str = Field(frozen=True) 

class ScoringConstants(BaseModel):
    ROOT_DIR_NAME: str = Field(frozen=True)
    SCORES_ROOT_DIR_NAME: str = Field(frozen=True)

__all__ = ["DataIngestionConstants", "DataTransformationConstants", "Constants", "JobDescriptionConstants", "ScoringConstants"]

##### values

In [ ]:
from src.ats.exception import CustomException

In [ ]:
# 3 things needed to be updated at a time [avl_cons, process, Note: Available name] 
# Note is inside docstring of function 'load' from this file and 'load_constants' inside __int__.py 
# update __all__ 

# from .schema import *
# from ..exception import CustomException 
from typing import List, Tuple, Dict
from datetime import datetime
from box import ConfigBox
import sys 


def __ing__(CONFIG:ConfigBox) -> Constants:
    return DataIngestionConstants(
        TIME_STAMP = datetime.now(),
        ROOT_DIR_NAME = CONFIG.ROOT_DIR, 
        DATA_ROOT_DIR_NAME = CONFIG.DATA.ROOT_DIR, 
        INGESTION_ROOT_DIR_NAME = CONFIG.DATA.INGESTION.ROOT_DIR, 
        RAW_DATA_DIR_NAME = CONFIG.DATA.INGESTION.RAW_DATA_DIR,
        OUTPUT_DIR_NAME = CONFIG.DATA.INGESTION.OUTPUT_DIR
    )

def __transform__(CONFIG:ConfigBox) -> Constants:
    return DataTransformationConstants(
        PROMPT = """
        You are a data structuring assistant. Extract and structure ONLY the information explicitly provided in the input data.

        IMPORTANT RULES:
        1. Extract ONLY information that is explicitly present in the data
        2. Do NOT add, infer, or generate any information
        3. If information is missing, leave fields as null/None
        4. For dates, use the exact format provided (e.g., "March 2019", "Present")
        5. For lists, only include items explicitly mentioned

        Input data to structure:
        {input_data}
        """, 
        TIME_STAMP = datetime.now(),
        ROOT_DIR_NAME = CONFIG.ROOT_DIR , 
        DATA_ROOT_DIR_NAME = CONFIG.DATA.ROOT_DIR , 
        TRANSFORMATION_ROOT_DIR_NAME = CONFIG.DATA.TRANSFORMATION.ROOT_DIR , 
        PARSED_DATA_DIR_NAME = CONFIG.DATA.TRANSFORMATION.PARSED_DATA_DIR ,
        STRUCTURED_DATA_DIR_NAME = CONFIG.DATA.TRANSFORMATION.STRUCTURED_DATA_DIR ,
        TRAIN_DATA_DIR_NAME = CONFIG.STRUCTURED_TRAINING.ROOT_DIR,
        OUTPUT_DIR_NAME = CONFIG.DATA.TRANSFORMATION.OUTPUT_DIR
    )

def __jd__(CONFIG:ConfigBox) -> Constants:
    return JobDescriptionConstants(
        TIME_STAMP = datetime.now(),
        ROOT_DIR_NAME = CONFIG.ROOT_DIR,
        JD_ROOT_DIR_NAME = CONFIG.JD.ROOT_DIR
    )

def __scoring__(CONFIG:ConfigBox) -> Constants:
    return ScoringConstants(
        ROOT_DIR_NAME = CONFIG.ROOT_DIR,
        SCORES_ROOT_DIR_NAME = CONFIG.SCORINGS.ROOT_DIR
    )

dataingestion = "DataIngestion"
datatransformation = "DataTransformation"
jobdescription = "JobDescription"
scorings = "Scoring"

avl_cons = [
    dataingestion, 
    datatransformation, 
    jobdescription,
    scorings
]
process = {
    dataingestion:__ing__,
    datatransformation:__transform__,
    jobdescription:__jd__,
    scorings:__scoring__
} 

def load(config:ConfigBox, name: str | List[str] | Tuple[str]) -> Dict: 
    """loads respective constants for the given name

    Args:
        config (ConfigBox): configuration for the object
        name (str | List[str] | Tuple[str]): name of required object  

        Note: Available names --> DataIngestion, DataTransformation, JobDescription, Scoring

    Raises:
        CustomException: Error shows with file name, line no and error message

    Returns:
        Dict: key = name of object used to load given in variable \'name\', 
        
              value = Object of the name used to load,

              example:
              output = load(config, "DataIngestion")
              output = { "DataIngestion" : DataIngestionConstants } 
              data_ingestion_constants = output["DataIngestion"] 
    """
    reqs:List[str] = []
    try:
        # validate type   
        if isinstance(name, str):
            reqs.append(name) 
        elif isinstance(name, List) or isinstance(name, Tuple):
            reqs += name 
        else:
            ValueError(f"Unsupported type {{{type(name)}}} for variable {{name}}") 

        # validate values 
        for req in reqs:
            if req not in avl_cons:
                ValueError(f"Unknown value provided in variable \'name\', {req}, name can only have values from {avl_cons}") 

        # run respective functions and return the output 
        output = {}
        for req in reqs: 
            func = process[req] 
            output[req] = func(config)

        return output
    except Exception as e: 
        raise CustomException(e, sys) 
    

__all__ = ["load"]

# entity

In [ ]:
# update __all__ 

from pydantic import BaseModel
from datetime import datetime
from pathlib import Path 


class DataIngestion(BaseModel):
    TIME_STAMP: datetime
    ROOT_DIR_PATH: Path
    DATA_ROOT_DIR_PATH: Path
    INGESTION_ROOT_DIR_PATH: Path
    RAW_DATA_DIR_PATH: Path
    OUTPUT_DIR_PATH: Path

class DataTransformation(BaseModel):
    PROMPT: str
    TIME_STAMP: datetime
    ROOT_DIR_PATH: Path
    DATA_ROOT_DIR_PATH: Path
    TRANSFORMATION_ROOT_DIR_PATH: Path
    PARSED_DATA_DIR_PATH: Path
    STRUCTURED_DATA_DIR_PATH: Path
    TRAIN_DATA_DIR_PATH: Path
    OUTPUT_DIR_PATH: Path

class JobDescription(BaseModel):
    TIME_STAMP: datetime
    ROOT_DIR_PATH: Path
    JD_ROOT_DIR_PATH: Path

class Scoring(BaseModel):
    ROOT_DIR_PATH: Path
    SCORES_ROOT_DIR_PATH: Path

__all__ = ["DataIngestion", "DataTransformation", "JobDescription", "Scoring"]

In [ ]:
from src.ats.components.scorers import *

In [9]:
import json
with open(
    "/home/hasan/Artificial-Intelligence/projects/Resume-ATS-Score-Checker/artifacts/data/transformation/structured/1(1)_pdf.json"
    ) as fp:
    resume_data = json.load(fp)
with open(
    "/home/hasan/Artificial-Intelligence/projects/Resume-ATS-Score-Checker/artifacts/job/28_09_2025_16_02_13.json"
    ) as fp:
    job_data = json.load(fp)

In [3]:
print("================================================ RD ================================================")
resume_data

================================================ RD ================================================


{'personal_info': {'name': 'Michael Brown',
  'email': 'michael.brown@example.com',
  'phone': '(111) 222-3333',
  'location': 'Chicago, IL',
  'linkedin': None},
 'professional_summary': {'headline': None,
  'summary': 'Recent Computer Science graduate with a strong foundation in programming and problem-solving. Eager to contribute to a dynamic team and grow as a software developer.',
  'total_experience_years': None,
  'career_level': None},
 'work_experience': [{'title': 'Software Development Intern',
   'company': 'Tech Innovations Ltd.',
   'start_date': 'May 2024',
   'end_date': 'August 2024',
   'duration_months': None,
   'responsibilities': ['Assisted in developing and testing software modules using Python.',
    'Participated in daily stand-ups and code reviews.',
    'Learned about agile development methodologies.'],
   'achievements': None,
   'technologies_used': ['Python']}],
 'skills': {'technical': ['Python'], 'soft': None, 'certifications': None},
 'education': [{'deg

In [4]:
print("================================================ JD ================================================")
job_data

================================================ JD ================================================


{'job_title': 'Data Scientist',
 'company_name': 'Capgemini',
 'location': 'Bengaluru',
 'job_type': 'Full Time, Permanent',
 'experience_level': '4 - 7 years',
 'job_description': 'About The Role\nThis role involves the development and application of engineering practice and knowledge in the following technologiesStandards and protocols, application software and embedded software for wireless and satellite networks, fixed networks and enterprise networks; connected devices (IOT and device engineering), connected applications (5G/ edge, B2X apps); and Telco Cloud, Automation and Edge Compute platforms. This role also involves the integration of network systems and their operations, related to the above technologies.\n\nAbout The Role - Grade Specific\nFocus on Connectivity and Network Engineering. Develops competency in own area of expertise. Shares expertise and provides guidance and support to others. Interprets clients needs. Completes own role independently or with minimum supervis

In [7]:
scorer = ResumeScorer()

In [10]:
scores = await scorer.score(resume_data, job_data)
scores

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'overall_score': 21.153300360934907,
 'score_breakdown': {'semantic_similarity': 44.06804275512695,
  'keyword_overlap': 9.090909090909092,
  'tfidf_similarity': 3.9940526580569977,
  'experience_match': 0.0},
 'match_quality': 'Poor',
 'model_used': 'all-roberta-large-v1-hybrid',
 'recommendation': 'Consider updating resume with more relevant technical keywords'}